# Computerphysik Programmiertutorial 4a
Prof. Dr. Matteo Rizzi und Dr. Markus Schmitt - Institut für Theoretische Physik, Universität zu Köln
&nbsp;

**ILIAS**: [https://www.ilias.uni-koeln.de/ilias/goto_uk_crs_3862489.html](https://www.ilias.uni-koeln.de/ilias/goto_uk_crs_3862489.html)

**Github**: [https://github.com/markusschmitt/compphys2021](https://github.com/markusschmitt/compphys2021)

**Inhalt dieses Notebooks**: Zeilennumerierung anzeigen, Array-Abstraktionen, Anonyme Funktionen, Rechnen auf dem Rechner: Maschinengenauigkeit, [E] Multiple Dispatch


## Zeilennumerierung anzeigen

Insbesondere beim Lesen von Fehlermeldungen ist es hilfreich sich die Zeilennumerierung innerhalb der Jupyter Notebook Zellen anzeigen zu lassen. Das geht über das Menü `View -> Toggle Line Numbers` oder den Shortcut `L`.

## Array-Abstraktionen (Array Comprehension)

Mit Array-Abstraktionen können Arrays oder andere iterierbare Datenstrukturen einfach "weiterverarbeitet" werden um daraus neue Arrays zu erzeugen.

Syntax:
```julia
neues_array = [<Anweisung> for <Variable> in <iterierbare Datenstruktur> if <Bedingung>]
```

Beispiel:

## Anonyme Funktionen

Bisher haben wir die Funktionendefinition der Form
```julia
function <Funktionenname>(<Argumente>)
    <Anweisungsblock>
    return <Rückgabewert>
end
```
kennengelernt. So wird jeder Funktion insbesondere ein *Name* zugewiesen. Alternativ kann man aber auch **anonyme Funktionen** definieren.

Anonyme Funktionen können einer Variablen zugewiesen werden:

Anonyme Funktionen können als Argument an andere Funktionen übergeben werden:

Genauso können anonyme Funktionen mit mehreren Argumenten definiert werden

## Rechnen auf dem Rechner: Maschinengenauigkeit

Zahlen werden im Computer in einem Binärcode dargestellt und für jede Zahl steht nur eine begrenzte Anzahl von Bits zur Verfügung. Es können daher weder alle ganzen Zahlen $\mathbb Z$ noch alle reellen Zahlen $\mathbb R$ dargestellt werden.

### Ganze Zahlen - `Int`

Wir haben schon in einem früheren Tutorial gesehen, dass Ganzzahlen in 64 bits als Binärzahlen dargestellt werden. Das ergibt automatisch eine Grenze für die größte darstellbare Zahl. Schauen wir uns diese Grenze an:

Die ganzen Zahlen auf dem Computer sind ein "Kreis":

In [ ]:
start=2^63-4
for i in 1:6
    println("$start + $i = $(start+i)")
end

### Fließkommazahlen - `Float`

Auf dem Computer können wir sehr leicht große Summen ausrechnen. Ein Beispiel ist die **Harmonische Reihe**

$$H_n=\sum_{k=1}^n\frac{1}{k}$$

Schreiben wir eine Funktion, die die $n$-te Harmonische Zahl $H_n$ berechnet:

In [ ]:
function H_forward(n, mytype=Float32)
    S = mytype(0.0)
    for k in 1:n
        S += mytype(1.0)/k
    end

    return S
end
    
H_forward(1000)

Da Addition kommutativ ist, können wir die Summe in beliebiger Reihenfolge ausrechnen, z.B. auch in umgekehrter Reihenfolge:

In [ ]:
function H_backward(n, mytype=Float32)
    S = mytype(0.0)
    for k in n:-1:1
        S += mytype(1.0)/k
    end

    return S
end
    
H_backward(1000)

Mit unterschiedlicher Reihenfolge der Summation erhalten wir unterschiedliche Ergebnisse!

Reelle Zahlen werden im Computer als **Fließkommazahlen** behandelt. Das bedeutet, dass sie bezüglich einer festen **Basis** $b$ in **Vorzeichen** $\pm$, **Mantisse** $m$ und **Exponent** $e$ zerlegt werden. Eine reelle Zahl $r\in\mathbb R$ wird also geschrieben als

$$r = \pm m\times b^e$$

Das Vorzeichen wird in einem Bit kodiert, für Mantisse und Exponent steht jeweils eine feste Zahl weiterer Bits zur Verfügung. Das Kodieren der Mantisse in einer begrenzten Anzahl von Bits bedeutet, dass wir bei jeder Zahl nur eine feste Anzahl von **signifikanten Ziffern** kennen. Die begrenzte Anzahl von Bits für den Exponenten bedeutet, dass es wie bei Ganzzahlen auch eine größte und kleinste darstellbare Fließkommazahl gibt.

Da der Computer nur mit einer bestimmten Zahl von signifikanten Ziffern rechnet, ist der Unterschied zwischen Zahlen nur begrenzt auflösbar. Diese "Auflösung" können wir experimentell bestimmen, indem wir fragen was die kleinste Zahl $\epsilon$ ist, so dass auf dem Computer $1.0+\epsilon>1.0$:

Die 64 bits des Datentyps `Float64` sind wie folgt aufgeteilt (Bild gestohlen von [benjaminjurke.com](https://benjaminjurke.com/content/articles/2015/loss-of-significance-in-floating-point-computations/)):

<img src="https://benjaminjurke.com/assets/images/articles/double64_bit_sequence.png">

Wir haben also 1 Bit für das Vorzeichen, 11 Bits kodieren den Exponenten als ganze Zahl zwischen $-1022$ und $1023$. Als Basis wird $b=2$ verwendet. Die darstellbaren Zahlen bewegen sich also (in etwa) zwischen $2^{-1022}\approx10^{-308}$ und $2^{1023}\approx10^{308}$. Die übrigen 52 Bits werden verwendet um die Mantisse als 

$$m=1+\sum_{n=1}^{52}\text{bit}_n\frac{1}{2^n}$$

zu kodieren.

Die folgende Funktion stellt eine gegebnene Fließkommazahl entsprechend dar.

In [ ]:
using Printf

function maschinendarstellung(x::Float64)
    
    bits = bitstring(x)
    sgn = bits[1]
    exponent = bits[2:12]
    mantissa = bits[13:64]
    
    println("Dezimal               | Vorz.  Exponent     Mantisse")
    println(@sprintf("%.15e |   %s    %s  %s", x, sgn, exponent, mantissa))
    return nothing
end

Schauen wir uns also an wie $1+\epsilon=1$ zustande kommt:

**Beim Addieren zweier Zahlen unterschiedlicher Größenordnung geht Information über die kleinere Zahl verloren.** Summationen sollten also immer so durchgeführt werden, dass nur ähnlich große Zahlen miteinander addiert werden.

Zurück zur Harmonischen Reihe. Welcher Summationsreihenfolge können wir also trauen? Für großes $n$ gilt

$$H_n\approx\log(n)+\gamma+\frac{1}{2n}-\frac{1}{12n^2}+\frac{1}{120n^4}$$

mit der Euler-Gamma Konstante $\gamma$. Wir können also unsere beiden Ergebnisse damit vergleichen:

In [ ]:
using PyPlot

function H_approx(n)
    n = Float64(n)
    return log(n)+Base.MathConstants.eulergamma+1.0/(2n)-1.0/(12n^2)+1.0/(120n^4)
end

n_werte = [2^n for n in 1:20]

Hn_fwd = [H_forward(2^n) for n in 1:20]
Hn_bwd = [H_backward(2^n) for n in 1:20]
Hn_approx = [H_approx(2^n) for n in 1:20]

semilogx(n_werte, abs.(Hn_fwd.-Hn_approx), "-o", label="forward")
semilogx(n_werte, abs.(Hn_bwd.-Hn_approx), "-o", label="backward")
xlabel("n")
ylabel("Differenz")
legend()

## [E] Multiple dispatch

Durch **multiple dispatch** können wir Funktionen definieren, deren Verhalten vom *Typ* der Argumente abhängt. Dazu werden Funktionen mit identischem Namen definiert, bei denen der Typ der Argumente durch Anhängen von `::<Datentyp>` spezifiziert ist.

Beispiel:

In [ ]:
function fun(x::Int64)
    println("Mein Argument ist vom Typ Int64.")
    println("     check: ", typeof(x))
end

function fun(x::Float64)
    println("Mein Argument ist vom Typ Float64.")
    println("     check: ", typeof(x))
end

Neben den konkreten Datentypen wie `Int64` oder `Float64` gibt es in Julia auch **abstrakte Datentypen**, durch die alle Datentypen hierarchisch strukturiert werden. Abstrakte Datentypen fassen die konkreten Datentypen in Gruppen zusammen. Beispiele sind `Integer` für ganze Zahlen oder `Number` für alle Zahlen. Ob ein konkreter Datentyp einem abstrakten Datentyp zugeordnet ist, kann man mit dem Operator `<:` überprüfen:

Funktionen können auch für abstrakte Datentypen *spezialisiert* werden:

In [ ]:
function more_fun(x::Integer)
    println("$x ist eine ganze Zahl vom Typ ", typeof(x)) 
end

more_fun(3)
more_fun(Int32(3))

So können wir z.B. unsere Funktion `maschinendarstellung()` von oben auch für den Datentyp `Float32` *spezialisieren*:

In [ ]:
using Printf

function maschinendarstellung(x::Float32)
    
    bits = bitstring(x)
    sgn = bits[1]
    exponent = bits[2:9]    # 8 bits für Exponent
    mantissa = bits[10:32]  # 23 bits für Mantisse
    
    println("Dezimal       | Vorz.  Exponent  Mantisse")
    println(@sprintf("%.7e |   %s    %s  %s", x, sgn, exponent, mantissa))
    return nothing
end